In [54]:
import numpy as np
import pandas as pd
import glob



path = "house-office-expenditures-with-readme"
#path =r'D:\Auburn University\6 - Fall 2018\1- INSY 6500 Information Technology for Operations\Homeworks\Project\data'
allFiles = glob.glob(path + "/*.csv")

list = [pd.read_csv(file,index_col = None, header = 0, encoding = 'unicode_escape') for file in allFiles]
    
df = pd.concat(list, axis = 0, ignore_index = True)
df = df[["BIOGUIDE_ID", "OFFICE","QUARTER","CATEGORY","DATE","PAYEE","START DATE","END DATE","PURPOSE","AMOUNT","YEAR", "TRANSCODE"]]
#df.sort_values(by="QUARTER", ascending=True )
df.head()


/Users/arminkhayyer/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,BIOGUIDE_ID,OFFICE,QUARTER,CATEGORY,DATE,PAYEE,START DATE,END DATE,PURPOSE,AMOUNT,YEAR,TRANSCODE
0,NaN,COMMUNICATIONS,2010Q1,OTHER SERVICES,NaN,03Â­10 P2 MFP0003226 ...,03/02/10,03/02/10,NON-TECHNOLOGY SERVICE CONTRCT,455.00,FISCAL YEAR 2010,NaN
1,NaN,COMMUNICATIONS,2010Q1,SUPPLIES AND MATERIALS,NaN,02Â­05 P2 MFP0003219 ALLSTEEL,11/28/09,11/28/09,HABITATION EXPENSES,47.26,FISCAL YEAR 2010,NaN
2,NaN,COMMUNICATIONS,2010Q1,SUPPLIES AND MATERIALS,NaN,03Â­05 P2 OSM42304 CDW GOVERN...,12/21/09,12/21/09,OFFICE SUPPLIES OUTSIDE,250.00,FISCAL YEAR 2010,NaN
3,NaN,COMMUNICATIONS,2010Q1,SUPPLIES AND MATERIALS,NaN,03Â­05 P2 OSM42304 DO,12/21/09,12/21/09,OFFICE SUPPLIES OUTSIDE,436.00,FISCAL YEAR 2010,NaN
4,NaN,COMMUNICATIONS,2010Q1,SUPPLIES AND MATERIALS,NaN,03Â­05 P2 OSM42304 DO,12/21/09,12/21/09,OFFICE SUPPLIES OUTSIDE,37.90,FISCAL YEAR 2010,NaN


In [55]:
#start date cleaning

df = df.loc[df["START DATE"] != '   ']
df['START DATE'].replace('', np.nan, inplace=True)


df.dropna(subset=['START DATE'], inplace=True)
df["START DATE"] = pd.to_datetime(df["START DATE"])

In [56]:

df = df.loc[df["END DATE"] != '   ']
df['END DATE'].replace('', np.nan, inplace=True)


df.dropna(subset=['END DATE'], inplace=True)
df["END DATE"] = pd.to_datetime(df["END DATE"])

question 1. What is the most expensive payee?
we have several payees,it's not good to just group buy that. what else we can do we can find the purpose of the largest amount payed. 

In [66]:
#df.groupby("PAYEE")["AMOUNT"].max()

def convert_to_float(row):
    string= str(row["AMOUNT"]).replace(",", "")
    return float(string)
    

df["AMOUNT"] = df.apply(lambda row: convert_to_float(row), axis =1)



In [133]:
# the data set is for 2015 second quarter, so there is no data point for 2016 in this dataset 
#df.to_csv("concatenated.csv")
new_df = df.loc[(df["CATEGORY"] =="PERSONNEL COMPENSATION"), :]
new_df.groupby(["PAYEE", "OFFICE"]).mean()


def split_strings(row):
    string= row["PAYEE"].rsplit( "  ", maxsplit=3)
    
    return string[-1]

df.dropna()
df.dropna(subset=['PAYEE'], inplace=True)
df["PAYEE"] = df.apply(lambda row: split_strings(row), axis= 1)



#df.head()


question 2. Which year was the most expensive year for the House?

question 3. Graphic How expenses changed during time?

question 4. Which office was the most expensive in each year?

question 5. Which quarter of each year was the most expensive?

question 6. What is the most commun purpose to spend money in House?

question 7. How much was the total expenditure for each purpose?

question 8. Graphic of total expenses in each year.

question 9. Graphic of total expensen in each year considering quarters.

question 10. Graphic of comparing the expense of each office during year... (can be lines for each office)
11. Changes in aggregate House expenditures (arranged by 2009 expenditure size)
12. Changes in staffing force, by position (for 20 most expensive positions by total expenditures, arranged from biggest decline to biggest gain)
13. 